## Sistemas de Recomendación

En este lab vamos a emplear la librería surprise. Esta es una librería que se basa en la API de scikit-learn y permite implementar varios algoritmos básicos de recomendación.



In [3]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

Comencemos cargando un dataset clásico en sistemas de recomendación: MovieLens (https://movielens.org/). Esta es una página de recomendación de películas que abrió información histórica. 

In [4]:
import pandas as pd

In [6]:
mlens = pd.read_csv("u.data",sep="\t",header=None)
mlens.columns = ["user_id","item_id","rating","timestamp"]

In [7]:
mlens = mlens.drop("timestamp", axis=1)

El paquete surprise no recibe directamente un objeto DataFrame sino que tiene para parsear y leer un conjunto de datos debe hacerlo a través de dos nuevos objetos: Reader y Dataset. En Reader debemos especificar el valor mínimo y el valor máximo de los ratings y Dataset nos permite leer datos desde distintas fuentes.

In [8]:
from surprise import Dataset, Reader
reader = Reader(rating_scale=(mlens["rating"].min(),mlens["rating"].max()))

In [9]:
dataset = Dataset.load_from_df(mlens,reader)

In [10]:
dataset

Ahora cargue SVD y GridSearchCV, ambos de surprise. Nota: GridSearchCV no está en surprise.GridSearchCV, surprise.GridSearch está deprecado.

In [16]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

Genere una grilla de parámetros donde se prueben distintas combinaciones de:
    - epochs: es la cantidad de pasadas sobre el dataset que hará el algoritmo empleando descenso por el gradiente
    - biased: usar parámetros de sesgo o no
    - lr_all: learning rate para todos los parámetros
    - reg_all: término de regularización para todos los parámetros (lambda)

In [17]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}

Emplee GridSearchCV, SVD y el diccionario con los parámetros para probar, y entrene un modelo. Note que a GridSearchCV necesita pasarle un modelo sin instanciar. Además, setee el parámetro refit a True y con measures = ["rmse","fcp"]

In [18]:
gs = GridSearchCV(SVD, param_grid, measures=['fcp',"rmse"], cv=3, refit=True)

In [19]:
gs.fit(dataset)

Imprima el rmse y el fcp, y la mejor combinación de parámetros

In [20]:
gs.best_score

{'fcp': 0.6979325575737213, 'rmse': 0.9639804176581834}

In [21]:
gs.best_params

{'fcp': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6},
 'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}

Guarde el modelo con mayor fcp y prediga el rating para el user id 196 e item id 242

In [22]:
best_model = gs.best_estimator["fcp"]

In [23]:
best_model.predict("196", "242")

Prediction(uid='196', iid='242', r_ui=None, est=3.52986, details={'was_impossible': False})

Pruebe empleando otros modelos como SVDpp, NMF, KNNWithZScore e intente superar el valor obtenido

In [24]:
from surprise import SVDpp
gs = GridSearchCV(SVDpp, param_grid, measures=['fcp',"rmse"], cv=3, refit=True)
gs.fit(dataset)
gs.best_score

{'fcp': 0.6990762088761239, 'rmse': 0.9640774759876952}